# Bekerja dengan Komputasi

Saat Anda menjalankan skrip sebagai pekerjaan Azure Machine Learning, Anda perlu menentukan konteks eksekusi untuk pekerjaan yang dijalankan. Salah satu konfigurasi utama adalah target komputasi tempat skrip akan dijalankan. Ini bisa menjadi stasiun kerja lokal (dalam hal ini instans komputasi), atau target komputasi jarak jauh seperti kluster komputasi terkelola Azure Machine Learning yang disediakan sesuai permintaan.

Dalam buku catatan ini, Anda akan membuat kluster komputasi dan menjelajahi target komputasi untuk pekerjaan.

## Sebelum Anda memulai

Anda memerlukan versi terbaru paket  **azureml-ai-ml** untuk menjalankan kode di notebook ini. Jalankan sel di bawah ini untuk memverifikasi bahwa sel diinstal.

> **Catatan:**
> Jika paket **azure-ai-ml** tidak diinstal, jalankan `pip install azure-ai-ml` untuk menginstalnya.

In [ ]:
## Menyambungkan ke ruang kerja Anda

Dengan paket SDK yang diperlukan terinstal, sekarang Anda siap untuk terhubung ke ruang kerja Anda.

Untuk menyambungkan ke ruang kerja, kita memerlukan parameter pengidentifikasi - ID langganan, nama grup sumber daya, dan nama ruang kerja. Nama grup sumber daya dan nama ruang kerja sudah diisi untuk Anda. Anda hanya memerlukan ID langganan untuk menyelesaikan perintah.

Untuk menemukan parameter yang diperlukan, klik nama langganan dan ruang kerja di kanan atas Studio. Panel akan terbuka di sebelah kanan.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Salin ID langganan dan ganti **YOUR-SUBSCRIPTION-ID** dengan nilai yang Anda salin. </p>

## Membuat klaster komputasi

Dalam banyak kasus, sumber daya komputasi lokal Anda mungkin tidak cukup untuk memproses eksperimen kompleks atau jangka panjang yang perlu memproses data dalam volume besar; dan Anda mungkin ingin memanfaatkan kemampuan untuk membuat dan menggunakan sumber daya komputasi secara dinamis di cloud. Azure Machine Learning mendukung berbagai target komputasi, yang dapat Anda tentukan dalam alur kerja Anda dan digunakan untuk menjalankan pekerjaan; membayar sumber daya hanya saat menggunakannya.

Anda dapat membuat kluster komputasi di [studio Azure Machine Learning](https://ml.azure.com), dengan menggunakan Python SDK, atau Azure CLI. Sel kode berikut memeriksa ruang kerja Anda untuk keberadaan nama `aml-cluster`kluster komputasi , dan jika tidak ada, membuatnya.

In [ ]:
Setelah membuat kluster komputasi, Anda hanya dapat mengubah konfigurasi untuk:

- `min_instances`: Jumlah minimum simpul
- `max_instances`: Jumlah maksimum simpul
- `idle_time_before_scale_down`: Waktu menganggur sebelum menurunkan skala

Saat ini, kluster komputasi `aml-cluster` Anda hanya dapat menskalakan melakukan maksimal satu simpul. Mari kita ubah menjadi dua, untuk memungkinkan komputasi paralel.

In [ ]:
Ketika kluster komputasi diperbarui, Anda dapat memverifikasi konfigurasinya dengan mencetak atributnya.

## Membuat skrip untuk melatih model

Untuk melatih model, Anda akan terlebih dahulu membuat skrip **diabetes_training.py** di folder **src** . Skrip menggunakan file **diabetes.csv** di folder yang sama dengan data pelatihan.

In [ ]:
## Menjalankan pekerjaan pada kluster komputasi

Sekarang, Anda siap untuk menjalankan pekerjaan pada kluster komputasi yang Anda buat.

> **Catatan:**
> Pekerjaan akan memakan waktu untuk memulai karena kluster komputasi perlu menskalakan dari nol ke satu simpul. Setelah kluster komputasi siap, skrip akan dijalankan. Setelah pekerjaan selesai, kluster komputasi akan menurunkan skala kembali ke nol simpul. Anda dapat meninjau status kluster komputasi di halaman **Komputasi** .

After you've created a compute cluster, you can only change the configuration for:

- `min_instances`: Minimum number of nodes
- `max_instances`: Maximum number of nodes
- `idle_time_before_scale_down`: Idle time before scale down

Currently, your compute cluster `aml-cluster` can only scale do a maximum of one node. Let's change that to two, to allow for parallel compute.

In [ ]:
from azure.ai.ml.entities import AmlCompute

cluster_scale = AmlCompute(
    name="aml-cluster",
    max_instances=2,
)
ml_client.begin_create_or_update(cluster_scale)

When the compute cluster is updated, you can verify its configuration by printing its attributes.

In [ ]:
cpu_cluster = ml_client.compute.get("aml-cluster")

print (
        f"AMLCompute with name {cpu_cluster.name} has a maximum of {cpu_cluster.max_instances} nodes"
    )

## Create a script to train a model

To train a model, you'll first create the **diabetes_training.py** script in the **src** folder. The script uses the **diabetes.csv** file in the same folder as the training data.

In [ ]:
%%writefile src/diabetes-training.py
# import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# load the diabetes dataset
print("Loading Data...")
diabetes = pd.read_csv('diabetes.csv')

# separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# set regularization hyperparameter
reg = 0.01

# train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))

## Run a job on a compute cluster

Now, you're ready to run the job on the compute cluster you created.

> **Note**:
> The job will take some time to start as the compute cluster will need to scale from zero to one node. Once the compute cluster is ready, the script will be run. When the job has finished, the compute cluster will scale back down to zero nodes. You can review the compute cluster's status in the **Compute** page.

In [ ]:
from azure.ai.ml import command

# configure job
job = command(
    code="./src",
    command="python diabetes-training.py",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="diabetes-train-cluster",
    experiment_name="diabetes-training"
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)